In [1]:
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 
using LsqFit
include("Elastic.jl")
using .Elastic
using FFMPEG

In [2]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 1

# 定义铜的FCC晶胞的基矢量
lattice_vectors = collect((Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))')

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,3,3,3)
fcell=filtercell(cpcell)
fig=visualize_unitcell_atoms(fcell)

[7484:24992:1005/222527.995:ERROR:t_skinfilemap.cpp(63)] SGRenderLog:CheckUpdate newMem.OpenMem Suc, szNewMem:Local\sgime_skin.Dpi144.v4.filemap.sogouime
[7484:24992:1005/222527.995:ERROR:t_skinfilemap.cpp(65)] SGRenderLog:CheckUpdate finish: old version:1
[7484:24992:1005/222527.995:ERROR:t_skinfilemap.cpp(67)] SGRenderLog:CheckUpdate finish: new version:4
[7484:24992:1005/222528.043:ERROR:t_skinfilemap.cpp(63)] SGRenderLog:CheckUpdate newMem.OpenMem Suc, szNewMem:Local\sgime_skin.Dpi144.v4.filemap.sogouime
[7484:24992:1005/222528.043:ERROR:t_skinfilemap.cpp(65)] SGRenderLog:CheckUpdate finish: old version:4
[7484:24992:1005/222528.044:ERROR:t_skinfilemap.cpp(67)] SGRenderLog:CheckUpdate finish: new version:4


In [3]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return 24*(2/rn^14-1/rn^8)*r
end

ct=5.0
interaction = Interaction(lj, Flj, ct, 0.1)

x=1:0.001:ct
y=interaction.cutenergy.(x)
lines(x,y)

In [4]:
el=Vector{Float64}([])
lattice_constant = 3.61
cl=1.3:0.02:2.5
en=0
fn=0
fl=Vector{Float64}([])
for lattice_constant in cl
    # 定义铜的FCC晶胞的基矢量
    lattice_vectors = (Matrix([
        lattice_constant 0.0 0.0; #a1
        0.0 lattice_constant 0.0; #a2
        0.0 0.0 lattice_constant] #a3
    ))
    # 创建铜的原子列表
    atoms = [Atom(pos) for pos in atom_positions]

    cell=UnitCell(lattice_vectors,atoms)
    cpcell=copycell(cell,3,3,3)
    fcell=filtercell(cpcell)
    en=cell_energy0(fcell,interaction,ifnormalize=true)
    push!(el,en)
    # fn=sum((cell_energy(cpcell,interaction)).^2)
    # push!(fl,fn)
end


In [39]:
# 创建一个新的 Figure
fig = Figure()

# 创建一个 Axis，并设置坐标范围
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )
vl=cl.^3*(cpcell.copy[1]*cpcell.copy[2]*cpcell.copy[3]);
# lines!(ax,cl,fl)
lines!(ax,cl,el)
# ylims!(ax, -0.001, 0.001)

fig

UndefVarError: UndefVarError: `cl` not defined

In [14]:
_,fig=BMfit(vl,el,cpcell)
fig

Fitted parameters:
V0 = 326.49223401016536
B0 = -3.88229067532352
B0' = 2.8167361588090265
E0 = -108.58895697259693
latticeconstant = 2.2952836537611065


In [5]:
thermostat = Thermostat(300.0, 10.0, 0.1, 0.0)
barostat=Barostat(15.0,cell.Volume,10.0,0.0)
z=cell2z(fcell,thermostat,barostat);

In [6]:
zl=[]
for i in 1:100
    z=RK3_step(z,0.001,fcell,interaction,thermostat,barostat)
    push!(zl,z)
end

In [9]:
zl[10]

652-element Vector{Float64}:
  0.015002653010977064
  0.015002653010977064
  0.015002653010977064
  0.015002653010977064
  0.015002653010977064
  2.514739030139311
  0.015002653010977064
  0.015002653010977064
  4.999473213291205
  0.015002653010977064
  ⋮
  0.00044035483524456514
  9.180638188953082e-5
  9.180638188953087e-5
  0.0004403548352445662
  9.18063818895292e-5
  9.180638188953151e-5
  9.180638188953095e-5
 -1.3468053731534997e-18
 -4.22599964745e-313

In [10]:
newcell=z2cell(zl[1000],fcell)
println(pressure_int(newcell,interaction))
fig=visualize_unitcell_atoms0(newcell)
fig

Warning the same atoms of atom 1 j=2 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=3 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=4 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=5 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=6 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=7 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=8 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=9 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=10 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=11 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=12 lead to the nan force,please check their cn
Warning the same atoms of atom 1 j=13 lead to the nan force,please check their cn
Warning the same atoms o

Excessive output truncated after 524324 bytes.


Warning the same atoms of atom 59 j=57 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=58 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=60 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=61 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=62 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=63 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=64 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=65 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=66 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=67 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=68 lead to the nan force,please check their cn
Warning the same atoms of atom 59 j=69 lead to the nan force,please check their cn
War

In [ ]:

# 假设func(i)是一个生成3D图的函数
function func(i)
    newcell=z2cell(zl[i],lattice_vectors)
    fig=visualize_unitcell_atoms0(newcell)
    return fig
end

function create_animation()
    video_file = "outputatom.mp4"
    n_frames = 1000
    fps = 1
    # 使用record函数生成动画
    record(fig, video_file, 1:n_frames; framerate = fps) do i
        func(i)
    end
end


# 调用函数生成视频
create_animation()